In [15]:
import pandas as pd
import numpy as np
from db import DBManager

# Downloading data

In [16]:
# query = """
#             SELECT
#                 dmt.date,
#                 dmt.avg_delay_no_cutoff,
#                 w.avg_tavg,
#                 w.avg_tmax,
#                 w.avg_tmin
#             FROM
#                 (SELECT date, AVG(delay_no_cutoff) AS avg_delay_no_cutoff
#                 FROM delay_max_test
#                 GROUP BY date) dmt
#             JOIN
#                 (SELECT date, AVG(tavg) AS avg_tavg, AVG(tmax) AS avg_tmax, AVG(tmin) AS avg_tmin
#                 FROM weather
#                 GROUP BY date) w
#             ON
#                 dmt.date = w.date
#             ORDER BY
#                 dmt.date;
#         """
# columns = ['date', 'avg_delay_no_cutoff', 'avg_tavg', 'avg_tmax', 'avg_tmin']

query = """
    SELECT
        dmt.date,
        dmt.fer,
        w.tavg,
        w.tmax,
        w.tmin,
        dmt.delay_no_cutoff
    FROM (
            SELECT date, fer, delay_no_cutoff
            FROM delay_max_test_v2
            ORDER BY date
        ) dmt
    JOIN (
            SELECT date, tavg, tmax, tmin
            FROM weather
            WHERE station = 'IC000004030'
        ) w
    ON
        dmt.date = w.date
    ORDER BY
        dmt.fer, w.tavg;
"""
columns = ['date', 'fer', 'tavg', 'tmax', 'tmin', 'delay_no_cutoff']

with DBManager() as db:
    res = db.query(query)
original_df = pd.DataFrame(res, columns=columns)
original_df

2024-01-06 13:25:10,450 INFO: Connected (version 2.0, client OpenSSH_8.9p1)
2024-01-06 13:25:10,740 INFO: Authentication (publickey) successful!
5432


,date,fer,tavg,tmax,tmin,delay_no_cutoff
0,2023-12-02,101-A,-3.9,-1.50,-5.50,8
1,2023-12-09,101-A,-2.2,1.40,-5.38,144
2,2023-12-10,101-A,-1.4,0.88,-4.62,370
3,2023-12-03,101-A,-1.4,1.16,-5.60,11
4,2023-12-30,101-A,-0.6,0.86,-4.70,37
...,...,...,...,...,...,...
16066,2023-10-09,A6-B,7.8,9.52,4.80,115
16067,2023-10-02,A6-B,8.3,9.80,5.48,125
16068,2023-10-20,A6-B,8.4,9.80,5.48,81
16069,2023-10-18,A6-B,10.0,11.58,7.42,103


# Extracting features from the data

In [17]:
df = original_df.copy()

In [18]:
# transform the date to day of the week, like monday, tuesday, etc.
df['day'] = pd.to_datetime(df['date'])
df['day'] = df['day'].dt.day_name()
df

,date,fer,tavg,tmax,tmin,delay_no_cutoff,day
0,2023-12-02,101-A,-3.9,-1.50,-5.50,8,Saturday
1,2023-12-09,101-A,-2.2,1.40,-5.38,144,Saturday
2,2023-12-10,101-A,-1.4,0.88,-4.62,370,Sunday
3,2023-12-03,101-A,-1.4,1.16,-5.60,11,Sunday
4,2023-12-30,101-A,-0.6,0.86,-4.70,37,Saturday
...,...,...,...,...,...,...,...
16066,2023-10-09,A6-B,7.8,9.52,4.80,115,Monday
16067,2023-10-02,A6-B,8.3,9.80,5.48,125,Monday
16068,2023-10-20,A6-B,8.4,9.80,5.48,81,Friday
16069,2023-10-18,A6-B,10.0,11.58,7.42,103,Wednesday


In [19]:
# one-hot encode the day of the week

# create a new DataFrame with dummy variables
df_dummies = pd.get_dummies(df['day'], prefix='day')

# concatenate the original DataFrame and the DataFrame with dummy variables
df = pd.concat([df, df_dummies], axis=1)

df

,date,fer,tavg,tmax,tmin,delay_no_cutoff,day,day_Friday,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday
0,2023-12-02,101-A,-3.9,-1.50,-5.50,8,Saturday,False,False,True,False,False,False,False
1,2023-12-09,101-A,-2.2,1.40,-5.38,144,Saturday,False,False,True,False,False,False,False
2,2023-12-10,101-A,-1.4,0.88,-4.62,370,Sunday,False,False,False,True,False,False,False
3,2023-12-03,101-A,-1.4,1.16,-5.60,11,Sunday,False,False,False,True,False,False,False
4,2023-12-30,101-A,-0.6,0.86,-4.70,37,Saturday,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16066,2023-10-09,A6-B,7.8,9.52,4.80,115,Monday,False,True,False,False,False,False,False
16067,2023-10-02,A6-B,8.3,9.80,5.48,125,Monday,False,True,False,False,False,False,False
16068,2023-10-20,A6-B,8.4,9.80,5.48,81,Friday,True,False,False,False,False,False,False
16069,2023-10-18,A6-B,10.0,11.58,7.42,103,Wednesday,False,False,False,False,False,False,True


In [20]:
# one-hot encode fer
df_dummies = pd.get_dummies(df['fer'], columns=['fer'], prefix='fer')
df = pd.concat([df, df_dummies], axis=1)
df

,date,fer,tavg,tmax,tmin,delay_no_cutoff,day,day_Friday,day_Monday,day_Saturday,...,fer_82-A,fer_88-A,fer_89-A,fer_A1,fer_A2,fer_A3,fer_A4,fer_A5,fer_A6,fer_A6-B
0,2023-12-02,101-A,-3.9,-1.50,-5.50,8,Saturday,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1,2023-12-09,101-A,-2.2,1.40,-5.38,144,Saturday,False,False,True,...,False,False,False,False,False,False,False,False,False,False
2,2023-12-10,101-A,-1.4,0.88,-4.62,370,Sunday,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2023-12-03,101-A,-1.4,1.16,-5.60,11,Sunday,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2023-12-30,101-A,-0.6,0.86,-4.70,37,Saturday,False,False,True,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16066,2023-10-09,A6-B,7.8,9.52,4.80,115,Monday,False,True,False,...,False,False,False,False,False,False,False,False,False,True
16067,2023-10-02,A6-B,8.3,9.80,5.48,125,Monday,False,True,False,...,False,False,False,False,False,False,False,False,False,True
16068,2023-10-20,A6-B,8.4,9.80,5.48,81,Friday,True,False,False,...,False,False,False,False,False,False,False,False,False,True
16069,2023-10-18,A6-B,10.0,11.58,7.42,103,Wednesday,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [21]:
df

,date,fer,tavg,tmax,tmin,delay_no_cutoff,day,day_Friday,day_Monday,day_Saturday,...,fer_82-A,fer_88-A,fer_89-A,fer_A1,fer_A2,fer_A3,fer_A4,fer_A5,fer_A6,fer_A6-B
0,2023-12-02,101-A,-3.9,-1.50,-5.50,8,Saturday,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1,2023-12-09,101-A,-2.2,1.40,-5.38,144,Saturday,False,False,True,...,False,False,False,False,False,False,False,False,False,False
2,2023-12-10,101-A,-1.4,0.88,-4.62,370,Sunday,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2023-12-03,101-A,-1.4,1.16,-5.60,11,Sunday,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2023-12-30,101-A,-0.6,0.86,-4.70,37,Saturday,False,False,True,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16066,2023-10-09,A6-B,7.8,9.52,4.80,115,Monday,False,True,False,...,False,False,False,False,False,False,False,False,False,True
16067,2023-10-02,A6-B,8.3,9.80,5.48,125,Monday,False,True,False,...,False,False,False,False,False,False,False,False,False,True
16068,2023-10-20,A6-B,8.4,9.80,5.48,81,Friday,True,False,False,...,False,False,False,False,False,False,False,False,False,True
16069,2023-10-18,A6-B,10.0,11.58,7.42,103,Wednesday,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [22]:
# copy the dataframe and save without any data, just the column names
copy = df.copy()

# remove all data from the dataframe
copy = copy.iloc[0:0]

# save the dataframe to a csv file
copy.to_csv('columns.csv')

# read the csv file back in
columns = pd.read_csv('columns.csv', index_col=0)
columns

,date,fer,tavg,tmax,tmin,delay_no_cutoff,day,day_Friday,day_Monday,day_Saturday,...,fer_82-A,fer_88-A,fer_89-A,fer_A1,fer_A2,fer_A3,fer_A4,fer_A5,fer_A6,fer_A6-B


# Training and evaluating with a RandomForestRegressor and 10-fold cross validation

In [24]:
# store all results from training in a list, so we can compare them later
all_results = []

In [51]:
import time
# do a five fold cross validation
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

CLASS = 'delay_no_cutoff'

X = df.drop([CLASS, 'date', 'fer', 'day'], axis=1)
y = df[CLASS]

print("Number of features: ", len(X.columns))

n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True)

# save the results
results = []

options = {
    # 'bootstrap':False, 
    # 'criterion':'squared_error', 
    'n_jobs':-1, 
    'n_estimators':200, 
    # 'verbose':1, 
    'max_features': 'sqrt',
}

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = RandomForestRegressor(**options)
    # model = LinearRegression()

    print("Fitting model...")
    tic = time.time()
    model.fit(X_train, y_train)
    toc = time.time()
    print("Time to fit: ", toc - tic)

    print("Predicting...")
    tic = time.time()
    y_pred = model.predict(X_test)
    toc = time.time()
    print("Time to predict: ", toc - tic)

    avg_mse = mean_squared_error(y_test, y_pred)
    print("MSE: ", avg_mse)
    avg_r2 = r2_score(y_test, y_pred)
    print("R2: ", avg_r2)
    # print("Coefficients: ", model.feature_importances_)
    # print("Coefficients: ", model.coef_)
    # print("Intercept: ", model.intercept_)

    print()

    results.append({
        'y_pred': y_pred,
        'y_test': y_test,
        'mse': avg_mse,
        'r2': avg_r2,
    })

all_results.append({
    "results": results,
    "options": options,
    "n_splits": n_splits,
    })


Number of features:  159
Fitting model...
Time to fit:  0.8126199245452881
Predicting...
Time to predict:  0.04009199142456055
MSE:  5535268.137588699
R2:  0.5400066712377729

Fitting model...
Time to fit:  0.8434841632843018
Predicting...
Time to predict:  0.04238176345825195
MSE:  4977310.576408106
R2:  0.5920955905125596

Fitting model...
Time to fit:  0.7609632015228271
Predicting...
Time to predict:  0.039247989654541016
MSE:  5091602.617937098
R2:  0.531869553734589

Fitting model...
Time to fit:  0.8370332717895508
Predicting...
Time to predict:  0.041053056716918945
MSE:  5935668.187509289
R2:  0.5224311368782483

Fitting model...
Time to fit:  0.7530727386474609
Predicting...
Time to predict:  0.04039287567138672
MSE:  5139773.800659737
R2:  0.5698125727141383



# Calculating average metrics across all folds

In [52]:
# get variance of target variable
variance = np.var(y.to_numpy())
print("Variance of target variable: ")
print(int(variance))


# get average results
print("Average results:")

avg_mse = 0
avg_r2 = 0

for r in results:
    avg_mse += r['mse']
    avg_r2 += r['r2']

avg_mse /= len(results)
avg_r2 /= len(results)

print("MSE: ", int(avg_mse))
print("R2: ", avg_r2)

Variance of target variable: 
11901639
Average results:
MSE:  5335924
R2:  0.5512431050154616


In [55]:
# when getting a good result, train on all data
model = RandomForestRegressor(**options)
model.fit(X, y)

RandomForestRegressor(max_features='sqrt', n_estimators=200, n_jobs=-1)

# Comparing with a dummy regressor that always predicts the mean of the training set (for each fer)

In [34]:
# create base results by predicting the mean of each fer
# y will be the values of each fer

def base():
    base_results = []
    for fer in df['fer'].unique():
        y_base = df[df['fer'] == fer][CLASS]
        y_pred = np.full(len(y_base), y_base.mean())
        mse = mean_squared_error(y_base, y_pred)
        r2 = r2_score(y_base, y_pred)
        if np.isnan(r2):
            r2 = 0
        base_results.append({
            'y_pred': y_pred,
            'y_test': y_base,
            'mse': mse,
            'r2': r2,
        })
    return base_results


base_results = base()

# get average mse and r2 for each fold
avg_base_mse = 0
avg_base_r2 = 0
for result in base_results:
    avg_base_mse += result['mse']
    avg_base_r2 = result['r2']
    avg_base_r2 += result['r2']

avg_base_mse /= len(base_results)
avg_base_r2 /= len(base_results)

print("Average MSE: ", avg_base_mse)
print("Average R2: ", avg_base_r2)


2024-01-06 13:29:27,596 WARNING: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.

Average MSE:  2791707.583108515
Average R2:  0.0


As we can see, our model is much better than just predicting the mean of the training set. 

# Saving the model to disk

In [35]:
# Save the model
import pickle
import os

# check if model already exists
if os.path.exists('model.pickle'):
    print("Model already exists. Delete it first if you want to overwrite...")
else:
    with open('model.pickle', 'wb') as f:
        pickle.dump(model, f)
        print("Model saved to model.pickle")

Model already exists. Delete it first if you want to overwrite...


In [39]:
# test loading the model
with open('model.pickle', 'rb') as f:
    m = pickle.load(f)

print("model type:", type(m))

# test the model
print("Testing the model...")
y_pred = m.predict(X_test)

print("MSE: ", mean_squared_error(y_test, y_pred))
print("R2: ", r2_score(y_test, y_pred))


model type: <class 'sklearn.ensemble._forest.RandomForestRegressor'>
Testing the model...
MSE:  2308467.557926831
R2:  0.7960573633710758


In [53]:
model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 200,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

# Upload the model to hopsworks

In [11]:
import hopsworks
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
import joblib
import os

In [9]:
project = hopsworks.login(project="ID2223_MKLepium")

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/186516


In [56]:
mr = project.get_model_registry()

model_dir = "bus_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

joblib.dump(model, model_dir + "/model.pkl")


input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)


bus_model = mr.python.create_model(
    name="bus_model", 
    metrics={"R2": avg_r2, "MSE": avg_mse},
    model_schema=model_schema,
    description="Bus Delay Prediction Model"
)

# Upload the model to the model registry, including all files in 'model_dir'
bus_model.save(model_dir)

Connected. Call `.close()` to terminate connection gracefully.


Uploading model files (0 dirs, 0 files):  17%|█▋        | 1/6 [00:00<00:01,  3.13it/s]








































































































































Uploading: 100.000%|██████████| 213760577/213760577 elapsed<01:24 remaining<00:00
Uploading input_example and model_schema:  33%|███▎      | 2/6 [01:24<03:19, 49.90s/it]

Uploading: 100.000%|██████████| 10966/10966 elapsed<00:01 remaining<00:00
Model export complete: 100%|██████████| 6/6 [01:32<00:00, 15.49s/it]                   

Model created, explore it at https://c.app.hopsworks.ai:443/p/186516/models/bus_model/1


Model(name: 'bus_model', version: 1)